In [1]:
%pylab inline

import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.stats as st
import hw1
import util
plt.rcParams['figure.dpi'] = 120
plt.rcParams['savefig.dpi'] = 300

Populating the interactive namespace from numpy and matplotlib


In [3]:
def convolution(image: np.ndarray, kernel: np.ndarray, kernel_width: int,
                kernel_height: int, add: bool, in_place: bool = False) -> np.ndarray:
    """Convolve an image with a kernel of size kernel_width*kernel_height. The weidth
    and height of kernel should be odd.
    If add is true, then 128 is added to each pixel of the result. If in_place is true,
    then the output is the copy of the input image, otherwise operations are
    performed on the original image."""
    
    # Creating an output with the same shape as image
    width = image.shape[0]
    height = image.shape[1]
    output = np.zeros([width, height])
    
    # Padding image with zeros
    padded_width = kernel_width // 2
    padded_height = kernel_height // 2
    pad_image = np.pad(image,((padded_height,padded_height),(padded_width,padded_width)))

    # Make sure the kernel size is odd
    if kernel_width % 2:
        print("Kernel width is not odd, adding a column of zeros")
        kernel = np.pad(kernel,((0,0),(0,1)))
        kernel_width += 1
    if kernel_height % 2:
        print("Kernel height is not odd, adding a row of zeros")
        kernel = np.pad(kernel,((0,1),(0,0)))
        kernel_height += 1

    # Convolution operation
    for x in range(0, width):
        for y in range(0, height):
            x1, y1 = np.add([x, y], [kernel_height, kernel_width])
            output[x,y] = np.sum(np.multiply(pad_image[x:x1, y:y1], kernel[::-1]))    
    
    if add:
        output = output + 128
        
    if in_place:
        return output
    else:
        image = output
        return image
    
"""
Task 2: GAUSSIAN BLUR
"""
def normalize_kernel(kernel: np.ndarray) -> np.ndarray:
    """ Normalize the kernel. Returns original kernel if the sum of elements is 0."""
    sum = np.sum(kernel)
    if sum == 0:
        return kernel
    else:
        result = kernel / np.sum(kernel)
        return result

def gaussian_blur_image(image: np.ndarray, sigma: float, in_place: bool = False) -> np.ndarray:
    """ Blur an image with the Gaussian filter. Sigma is the standard deviation
    of the Gaussian. Kernel is normalized before the convolution."""
    radius = int(math.ceil(3 * sigma))
    kernel_size = 2 * radius + 1
    kernel = np.zeros([kernel_size,kernel_size])
    
    multiplier = 1 / (2 * math.pi * sigma**2)
    divider = 2 * sigma**2
    origin = radius
    
    # Creating Gaussian filter
    # x and y represent distances from the origin = (radius, radius)
    for x in range(0,radius + 1):
        for y in range(0,radius + 1):
            x_r, y_r = np.add(origin, [x, y])
            x_l, y_l = np.subtract(origin, [x, y])
            kernel[x_r, y_r] = multiplier * math.exp(-(x**2 + y**2)/divider)
            kernel[x_r, y_l] = kernel[x_l, y_r] = kernel[x_l, y_l] = kernel[x_r, y_r]
    
    kernel = normalize_kernel(kernel)
    output = convolution(image,kernel,kernel_size,kernel_size,False,in_place)
    return output

"""
Task 3: SEPARABLE GAUSSIAN BLUR
"""
def separable_gaussian_blur_image (image : np.ndarray, sigma : float, in_place : bool = False) -> np.ndarray :
    """ Blur an image with the separable Gaussian filter. Sigma is the standard
    deviation of the Gaussian."""
    radius = int(math.ceil(3 * sigma))
    kernel_size = 2 * radius + 1
    
    # Creating two kernels
    h_kernel = np.zeros([1,kernel_size])
    v_kernel = np.zeros([kernel_size,1])
    
    multiplier = 1 / (math.sqrt(2 * math.pi) * sigma)
    divider = 2 * sigma**2
    origin = radius
    
    for x in range(0,radius + 1):
        x_r = origin + x
        x_l = origin - x
        h_kernel[0, x_r] = multiplier * math.exp(-x**2/divider)
        v_kernel[x_r, 0] = v_kernel[x_l, 0] = h_kernel[0, x_l] = h_kernel[0, x_r]

    # Normalizing kernels
    h_kernel = normalize_kernel(h_kernel)
    v_kernel = normalize_kernel(v_kernel)
    
    # Convolving the image in two steps
    output = convolution(image, h_kernel, kernel_size, 1, False, in_place)
    output = convolution(output, v_kernel, 1, kernel_size, False, in_place)
    
    return output

In [9]:
songfestival = cv2.imread("images/songfestival.jpg", cv2.IMREAD_GRAYSCALE)
songfestival = np.asarray(songfestival,dtype=np.double)


In [10]:
import time

In [11]:
t1 = time.perf_counter()
songfestival_blur1 = gaussian_blur_image(songfestival, 4.0)
print(time.perf_counter() - t1)
util.plotImage(util.img_to_uint8(songfestival_blur1))

Kernel width is not odd, adding a column of zeros
Kernel height is not odd, adding a row of zeros


ValueError: operands could not be broadcast together with shapes (26,25) (26,26) 

In [8]:
t2 = time.perf_counter()
songfestival_blur2 = separable_gaussian_blur_image(songfestival, 4.0)
print(time.perf_counter() - t2)
util.plotImage(util.img_to_uint8(songfestival_blur2))

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (2,2) and requested shape (3,2)